In [ ]:
import os
import pandas as pd
from openai import OpenAI

client = OpenAI(
    os.environ.get("OPENAI_API_KEY")
)

In [ ]:
def generate_question(article, prompt_list, level):
    result_list = []
    if level == 1:
      instruction = prompt_list[0]

    elif level == 2:
      instruction = prompt_list[1]

    elif level == 3:
      instruction = prompt_list[2]

    elif level == 4:
      instruction = prompt_list[3]

    chat_completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "你是一個能幹的閱讀理解問題生成器，始終遵循給定的說明和要求來生成問題。"},
             {"role": "user", "content": f"{instruction}\n文章:{article}"}
            ],
    max_tokens=200,
    temperature=0.6,
    top_p=0.9
    )
    output = chat_completion.choices[0].message.content
    print(output)
    result_list.append([article, output, level])
    return result_list

def process(article_data,prompt_path):
      prompt_list = []
      temp_prompt_data = pd.read_excel(prompt_path)
      for n in range(len(temp_prompt_data)):
          prompt_list.append(temp_prompt_data['prompt'][n])
      result_list = []
      for n in range(len(article_data)):
          print(f"{n+1}/50:")
          article = article_data['clean_article'][n]
          result_list.extend(generate_question(article, prompt_list, level=1))
          result_list.extend(generate_question(article, prompt_list, level=2))
          result_list.extend(generate_question(article, prompt_list, level=3))
          result_list.extend(generate_question(article, prompt_list, level=4))
      df = pd.DataFrame(result_list, columns = ['article', 'question', 'required level'])
      df.to_excel("GPT4o(zero).xlsx", index=None)

article_data = pd.read_excel("test article.xlsx")
prompt_path = "QAG_prompt_zero.xlsx"
process(article_data, prompt_path)